In [2]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
submission = pd.read_csv('./data/sample_submission.csv')

**train.csv**  
데이터 전처리 과정이 끝난 학습 데이터 (추가 데이터 전처리 과정을 진행하지 않습니다.)
- RF 모델 하이퍼파라미터를 제출 시, 해당 데이터로 자동적으로 학습됩니다.
- person_id: 유저별 고유 아이디
- Sex: 성별
- past_login_total: 과거(5월 8일 이전)에 로그인한 총 횟수
- past_1_month_login: 과거 1달간 로그인한 총 횟수
- past_1_week_login: 과거 1주간 로그인한 총 횟수
- sub_size: 과거에 데이콘 대회에서의 총 제출 수
- email_type: 가입한 이메일 종류
- phone_rat: 폰으로 접속한 비율
- apple_rat: 애플 기기로 접속한 비율
- login: 로그인 여부 

**sample_submission.csv** 
RF 모델을 학습시킬 모델 하이퍼파라미터 목록
- n_estimators:

기본값: 10

범위: 10 ~ 1000 사이의 양의 정수. 일반적으로 값이 클수록 모델 성능이 좋아지지만, 계산 비용과 시간도 증가합니다.

- criterion:

기본값: 'gini'

옵션: 'gini', 'entropy'. 'gini'는 진니 불순도를, 'entropy'는 정보 이득을 기준으로 합니다.
- max_depth:

기본값: None

범위: None 또는 양의 정수. None으로 설정하면 노드가 모든 리프가 순수해질 때까지 확장됩니다. 양의 정수를 설정하면 트리의 최대 깊이를 제한합니다.
- min_samples_split:

기본값: 2

범위: 2 이상의 정수 또는 0과 1 사이의 실수 (비율을 나타냄, (0, 1] ). 내부 노드를 분할하기 위해 필요한 최소 샘플 수를 지정합니다.
- min_samples_leaf:

기본값: 1

범위: 1 이상의 정수 또는 0과 0.5 사이의 실수 (비율을 나타냄, (0, 0.5] ). 리프 노드가 가져야 하는 최소 샘플 수를 지정합니다.
- min_weight_fraction_leaf:

기본값: 0.0

범위: 0.0에서 0.5 사이의 실수. 리프 노드에 있어야 하는 샘플의 최소 가중치 비율을 지정합니다.
- max_features:

기본값: 'auto'

옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수. 최적의 분할을 찾기 위해 고려할 특성의 수 또는 비율을 지정합니다. 'auto'는 모든 특성을 사용함을 의미하며, 'sqrt'와 'log2'는 각각 특성의 제곱근과 로그2를 사용합니다. None은 'auto'와 동일하게 모든 특성을 의미합니다.
- max_leaf_nodes:

기본값: None

범위: None 또는 양의 정수. 리프 노드의 최대 수를 제한합니다. None은 무제한을 의미합니다.
- min_impurity_decrease:

기본값: 0.0

범위: 0.0 이상의 실수. 노드를 분할할 때 감소해야 하는 불순도의 최소량을 지정합니다.
- bootstrap:

기본값: True

옵션: True, False. True는 부트스트랩 샘플을 사용하여 개별 트리를 학습시킵니다. False는 전체 데이터셋을 사용하여 각 트리를 학습시킵니다.

In [3]:
train.head()

,person_id,Sex,past_login_total,past_1_month_login,past_1_week_login,sub_size,email_type,phone_rat,apple_rat,login
0,0,0,3.0,1.0,1.0,0.0,0,0.000000,0.000000,1
1,1,1,111.0,26.0,7.0,2.0,0,0.072072,0.000000,1
2,3,1,13.0,13.0,11.0,7.0,0,0.076923,1.000000,1
3,4,1,28.0,12.0,5.0,0.0,0,0.071429,0.071429,1
4,5,1,4.0,4.0,4.0,0.0,2,0.000000,0.000000,1


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   person_id           1309 non-null   int64  
 1   Sex                 1309 non-null   int64  
 2   past_login_total    1309 non-null   float64
 3   past_1_month_login  1309 non-null   float64
 4   past_1_week_login   1309 non-null   float64
 5   sub_size            1309 non-null   float64
 6   email_type          1309 non-null   int64  
 7   phone_rat           1309 non-null   float64
 8   apple_rat           1309 non-null   float64
 9   login               1309 non-null   int64  
dtypes: float64(6), int64(4)
memory usage: 102.4 KB


#### 컬럼별 특징
- login : 타겟값, 불균형 심함 89:11
- person_id : 고유값이므로 제거
- sex : 54:46
- past_login_total : 0~203,  43% 1 / 11% 2 / 10% 0 , 이상치 처리
- past_1_month_login : 0~93, 85% 0, 이상치 처리
- past_1_week_login : 0~23, 90% 0, 이상치 처리
- sub_size : 0~358, 83% 0, 이상치 처리
- email_type : 61% 0 / 21% 1 / 13% 2
- phone_rat : 0~1, 76% 0
- apple_rat : 0~1, 73% 0

## 전처리

### 이상치 처리

### 타겟값 불균형 해결

### 스케일링